<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_08_Describe_Related_Images_from_Web_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to search the Web and ingest web pages, crawl image links, and describe the images. It also shows how to search the image descriptions based on the original Web search to find images related to your search text.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.


---

Install Graphlit Python client SDK

In [ ]:
!pip install --upgrade graphlit-client

Initialize Graphlit

In [ ]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Define Graphlit helper functions

In [15]:
from typing import List, Optional

# Create specification for OpenAI GPT-4o
async def create_openai_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="OpenAI GPT-4o",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=enums.OpenAIModels.GPT4O_128K,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create specification for Anthropic Sonnet 3.5
async def create_anthropic_specification():
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name="Anthropic Claude Sonnet 3.5",
        type=enums.SpecificationTypes.EXTRACTION,
        serviceType=enums.ModelServiceTypes.ANTHROPIC,
        anthropic=input_types.AnthropicModelPropertiesInput(
            model=enums.AnthropicModels.CLAUDE_3_5_SONNET,
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

# Create enrichment workflow to crawl only image links from web pages
async def create_workflow(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.WorkflowInput(
        name="Link Crawling",
        extraction=input_types.ExtractionWorkflowStageInput(
            jobs=[
                input_types.ExtractionWorkflowJobInput(
                    connector=input_types.EntityExtractionConnectorInput(
                        type=enums.EntityExtractionServiceTypes.MODEL_IMAGE,
                        modelImage=input_types.ModelImageExtractionPropertiesInput(
                            specification=input_types.EntityReferenceInput(id=specification_id)
                        )
                    )
                )
            ]
        ),
        enrichment=input_types.EnrichmentWorkflowStageInput(
            link=input_types.LinkStrategyInput(
                enableCrawling=True,
                allowContentDomain=True,
                allowedLinks=[enums.LinkTypes.FILE],
                allowedFiles=[enums.FileTypes.IMAGE],
            )
        )
    )

    try:
        response = await graphlit.client.create_workflow(input)

        return response.create_workflow.id if response.create_workflow is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_feed(search_text: str, read_limit: int, workflow_id: str):
    if graphlit.client is None:
        return;

    # NOTE: Uncomment one of these to select the search service
    service_type = enums.SearchServiceTypes.TAVILY
    #service_type = enums.SearchServiceTypes.EXA

    input = input_types.FeedInput(
        name="Web Search",
        type=enums.FeedTypes.SEARCH,
        search=input_types.SearchFeedPropertiesInput(
            type=service_type,
            text=search_text,
            readLimit=read_limit
        ),
        workflow=input_types.EntityReferenceInput(
            id=workflow_id
        )
    )

    try:
        response = await graphlit.client.create_feed(input)

        return response.create_feed.id if response.create_feed is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def is_feed_done(feed_id: str):
    if graphlit.client is None:
        return;

    response = await graphlit.client.is_feed_done(feed_id)

    return response.is_feed_done.result if response.is_feed_done is not None else None

# Locate content ingested by feed, optionally search by text
async def query_contents(feed_id: str, content_types: Optional[List[enums.ContentTypes]], file_types: Optional[List[enums.FileTypes]] = None, search: Optional[str] = None):
    if graphlit.client is None:
        return;

    try:
        response = await graphlit.client.query_contents(
            filter=input_types.ContentFilter(
                search=search,
                searchType=enums.SearchTypes.HYBRID,
                types=content_types,
                fileTypes=file_types,
                feeds=[
                    input_types.EntityReferenceFilter(
                        id=feed_id
                    )
                ]
            )
        )

        return response.contents.results if response.contents is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def delete_all_workflows():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_workflows(is_synchronous=True)


async def delete_all_feeds():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_feeds(is_synchronous=True)


Execute Graphlit example

In [16]:
from IPython.display import display, Markdown, Image
import time

# Remove any existing feeds and workflows; only needed for notebook example
await delete_all_workflows()
await delete_all_feeds()

print('Deleted all feeds.')

read_limit = 5 # how many search results to ingest from feed

search_text = "Seattle Kraken mascot Buoy"

# NOTE: uncomment just one of these to use different model
specification_id = await create_anthropic_specification()
#specification_id = await create_openai_specification()

feed_id = None

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    workflow_id = await create_workflow(specification_id)

    if workflow_id is not None:
        print(f'Created workflow [{workflow_id}].')

        feed_id = await create_feed(search_text, read_limit, workflow_id)

        if feed_id is not None:
            print(f'Created feed [{feed_id}].')

            # Wait for feed to complete, since ingestion happens asychronously
            done = False
            time.sleep(5)
            while not done:
                done = await is_feed_done(feed_id)

                if not done:
                    time.sleep(2)

            print(f'Completed feed [{feed_id}].')

Deleted all feeds.
Created specification [200bf925-dede-4d62-a161-27c1dcb6ae03].
Created workflow [6e0fc921-c5dc-44a1-b11b-2ec54c54d563].
Created feed [b5473fa9-73d6-457f-9968-cad93504de32].
Completed feed [b5473fa9-73d6-457f-9968-cad93504de32].


In [17]:
if feed_id is not None:
    # Show all the web pages
    contents = await query_contents(feed_id, [enums.ContentTypes.PAGE])

    if contents is not None:
        print(f'Found {len(contents)} web pages in feed [{feed_id}].')
        print()

        for content in contents:
            if content is not None:
                display(Markdown(f'URI: {content.uri}'))

    print()

    # Show all the images crawled from the web page
    contents = await query_contents(feed_id, [enums.ContentTypes.FILE], [enums.FileTypes.IMAGE])

    if contents is not None:
        print(f'Found {len(contents)} images in feed [{feed_id}].')
        print()

        for content in contents:
            if content is not None and content.image is not None:
                display(Image(url=content.image_uri, width=256))

    print()

    # Show just the images related to the search text, and the image text and descriptions
    contents = await query_contents(feed_id, [enums.ContentTypes.FILE], [enums.FileTypes.IMAGE], search_text)

    if contents is not None:
        print(f'Found {len(contents)} images in feed [{feed_id}] related to [{search_text}].')
        print()

        for content in contents:
            if content is not None:
                display(Markdown(f'## {content.name} [{content.id}]:'))
                display(Markdown(f'Original image size: {content.image.width}x{content.image.height}'))

                if content.markdown is not None:
                    display(Markdown(f'Text in image: {content.markdown}'))

                if content.image is not None:
                    display(Image(url=content.image_uri, width=512))

                    if content.image.description is not None:
                        display(Markdown(f'### Description:\n{content.image.description}'))

Found 4 web pages in feed [b5473fa9-73d6-457f-9968-cad93504de32].



URI: https://www.youtube.com/watch?v=yTgZQgrm8JA

URI: https://www.nhl.com/kraken/fans/team-mascot-buoy

URI: https://en.wikipedia.org/wiki/Buoy_(mascot)

URI: https://www.espn.com/nhl/story/_/id/34701087/how-seattle-kraken-kept-their-new-mascot-buoy-secret

...

### Description:
The image presents a digital collage artwork centered around the theme of the Seattle Kraken, a professional ice hockey team. The composition features a mix of photographic elements and graphic design, creating a visually striking promotional piece.

At the top left corner, there's a stylized logo of what appears to be a skull-like face with tentacle-like features, likely representing the Kraken mascot. This logo is rendered in shades of blue and white, embodying the team's color scheme and fierce persona.

The background of the image incorporates the Kraken's 'S' logo in a large, faded format, serving as a watermark-like element that ties the entire composition together. This subtle use of the team's branding reinforces the theme without overwhelming the other visual elements.

The main focus of the artwork is a collection of player images arranged across the bottom half of the composition. These figures are presented in various poses and uniforms, showcasing different aspects of the team's identity. Some players are shown in action on the ice, while others appear in more casual settings, possibly representing off-ice personalities or promotional shots.

The central figure in the image is a player wearing the number 3 jersey, prominently displayed and larger than the other elements. This positioning suggests this player may have a significant role within the team, possibly as a captain or key player.

Text is incorporated into the design, with '3 Kraken Captain Scenarios' displayed prominently at the top of the image in a bold, futuristic font that complements the overall aesthetic. This text suggests that the artwork is likely part of a discussion or presentation about potential team leadership.

The color palette of the artwork is dominated by shades of blue, teal, and white, which are consistent with the Seattle Kraken's official colors. This cohesive use of color helps to unify the various elements and reinforces the team's brand identity throughout the piece.

The overall composition creates a sense of depth and layering, with some elements appearing to be cut out and placed over others. This technique adds visual interest and helps to guide the viewer's eye across the different components of the artwork.

The combination of photographic elements with graphic design creates a dynamic and modern feel, likely appealing to sports fans and reflecting the excitement and energy associated with ice hockey. The artwork effectively balances team branding with player representation, creating a compelling visual narrative about the Seattle Kraken.

In conclusion, this digital collage serves as a powerful piece of sports marketing material, effectively combining team imagery, player representation, and graphic design elements to create an engaging visual story about the Seattle Kraken and potential team leadership scenarios.

## zdvth9z8rgrd95atnj0j.png [6803451e-bdf9-4454-af33-56e75c248ec3]:

Original image size: 720x120

Text in image: SEABUOY



### Description:
The image presents a sleek and modern logo design for a brand called 'SEABUOY'. The logo is composed of two main elements: a geometric symbol and the brand name text.

The symbol on the left side of the logo is a minimalist representation that resembles the letter 'X'. It's constructed from two intersecting lines that form four triangular shapes. This geometric design gives the logo a contemporary and abstract feel, potentially symbolizing precision, strength, or interconnectedness.

To the right of the symbol, the brand name 'SEABUOY' is displayed in bold, uppercase letters. The font choice is sans-serif, which contributes to the modern aesthetic of the overall design. The text is aligned with the symbol, creating a balanced and cohesive look.

The color scheme of the logo is simple yet effective. The 'X' symbol appears in white or light gray, standing out against a dark background. The text 'SEABUOY' is rendered in a deep navy blue color, which immediately evokes associations with the sea, aligning well with the brand name.

The contrast between the light symbol and the dark text creates visual interest and ensures that both elements of the logo are distinct and easily readable. This color choice also suggests a sense of professionalism and reliability.

The name 'SEABUOY' itself is intriguing, as it combines two nautical terms. 'Sea' obviously refers to the ocean, while 'buoy' is a floating device used for navigation or marking specific locations in bodies of water. This name suggests that the brand or company is likely involved in marine-related products, services, or technologies.

The simplicity of the logo design makes it versatile and scalable, suitable for various applications from small digital icons to large signage. Its clean lines and straightforward presentation ensure that it would remain recognizable even at smaller sizes.

Overall, this logo effectively communicates a brand identity that is modern, professional, and linked to maritime activities or industries. The combination of the abstract symbol and the clear, bold text creates a memorable visual identity that could represent anything from marine technology to oceanographic research or nautical equipment.